In [1]:
import pandas as pd
from pearlsim.ml_utilities import *
import os
import random
import numpy as np

In [2]:
current_data = pd.read_csv("training_data/current_target.csv", index_col=0).drop(columns=["power"])

In [3]:
file_names = os.listdir("training_data/")
concentrations = []
for name in file_names:
    if "bumat0" in name:
        concentrations += extract_from_bumat("training_data/"+name)


In [4]:
file_name = "burnup_models/burnup0.serpent"

input_s = generate_pebble_burnup_model("pebble_gFHR_template.serpent", current_data.iloc[10], 1120, concentrations[3], 959, 6)

with open(file_name, 'w') as f:
    f.write(input_s)

In [5]:
np.random.seed(1)
iterations = 1500
time_steps = [0.06525, 0.2, 1.5, 3.0, 4.6, 6.525, 6.525, 6.525]
conc_array = []
current_array = []
day_array = []
power_array = []



for i in range(1,iterations+1):
    conc = concentrations[np.random.randint(len(concentrations))]
    current = current_data.iloc[np.random.randint(len(current_data))]
    days = time_steps[ np.random.randint(len(time_steps)) ]
    power = np.random.randint(800, high=3000)
    power_array += [power]
    conc_array += [conc]
    current_array += [current]
    day_array += [days]
    file_name = f"burnup_models/burnup{i}.serpent"
    input_s = generate_pebble_burnup_model("pebble_gFHR_template.serpent", current, power, conc, 959,  days)

    #with open(file_name, 'w') as f:
    #    f.write(input_s)

num_cores = 20
iterations = 1000
for i in range(1,iterations+1):

    file_name = f"burnup{i}.serpent"
    os.system(f"sss2_2_0 {file_name} -omp {num_cores}")

In [6]:
features_df = pd.DataFrame({"depletion_time": day_array})
current_df = pd.DataFrame(current_array).reset_index(drop=True)
conc_df = pd.DataFrame(conc_array).fillna(0)
features_df = features_df.join(current_df)
features_df = features_df.join(conc_df)
features_df['power'] = power_array
features_df.iloc[0:5]

,depletion_time,2.9005e-08,9.9e-08,2.1e-07,4.525e-07,7.985000000000001e-07,1.061e-06,1.5025e-06,2.9275e-06,6.9385e-06,...,62147<lib>,62148<lib>,62149<lib>,62150<lib>,62151<lib>,621520,631510,92235<lib>,92238<lib>,power
0,4.600,2.126150e+14,5.195180e+14,5.748160e+14,3.973870e+14,1.264600e+14,2.376000e+13,8.956950e+13,1.416580e+14,1.624290e+14,...,4.414775e-07,1.170724e-06,9.256264e-08,0.000005,2.364126e-07,1.322357e-06,4.404614e-10,0.002711,0.018585,1896
1,3.000,2.301940e+14,5.653970e+14,6.157750e+14,4.178810e+14,1.131750e+14,2.829020e+13,1.121280e+14,1.649230e+14,1.915860e+14,...,2.036377e-07,3.743376e-07,9.149996e-08,0.000003,2.329736e-07,6.794943e-07,2.925087e-10,0.003390,0.018751,1760
2,4.600,1.361290e+14,3.365870e+14,3.607000e+14,1.822650e+14,2.672060e+13,1.511560e+12,2.970530e+13,2.870360e+13,5.100380e+13,...,9.148329e-07,3.011392e-06,8.438918e-08,0.000009,2.477724e-07,2.501804e-06,6.966229e-10,0.001834,0.018358,2002
3,6.525,4.433100e+14,9.815350e+14,9.476020e+14,5.650410e+14,1.337090e+14,5.945700e+13,1.086090e+14,1.727270e+14,2.245720e+14,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.004670,0.018973,1052
4,6.525,2.093670e+14,4.521590e+14,4.341860e+14,2.476010e+14,5.047690e+13,2.374190e+13,5.077140e+13,9.692880e+13,8.705620e+13,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.004670,0.018973,1362


In [7]:
features_df.to_csv("training_data/burnup_features.csv")

In [5]:
features_df = pd.read_csv("training_data/burnup_features.csv", index_col=0)


FileNotFoundError: [Errno 2] File b'training_data/burnup_features.csv' does not exist: b'training_data/burnup_features.csv'

In [8]:
display(features_df)

,depletion_time,2.9005e-08,9.9e-08,2.1e-07,4.525e-07,7.985000000000001e-07,1.061e-06,1.5025e-06,2.9275e-06,6.9385e-06,...,62147<lib>,62148<lib>,62149<lib>,62150<lib>,62151<lib>,621520,631510,92235<lib>,92238<lib>,power
0,4.600,2.126150e+14,5.195180e+14,5.748160e+14,3.973870e+14,1.264600e+14,2.376000e+13,8.956950e+13,1.416580e+14,1.624290e+14,...,4.414775e-07,1.170724e-06,9.256264e-08,5.379601e-06,2.364126e-07,1.322357e-06,4.404614e-10,0.002711,0.018585,1896
1,3.000,2.301940e+14,5.653970e+14,6.157750e+14,4.178810e+14,1.131750e+14,2.829020e+13,1.121280e+14,1.649230e+14,1.915860e+14,...,2.036377e-07,3.743376e-07,9.149996e-08,2.919479e-06,2.329736e-07,6.794943e-07,2.925087e-10,0.003390,0.018751,1760
2,4.600,1.361290e+14,3.365870e+14,3.607000e+14,1.822650e+14,2.672060e+13,1.511560e+12,2.970530e+13,2.870360e+13,5.100380e+13,...,9.148329e-07,3.011392e-06,8.438918e-08,9.242711e-06,2.477724e-07,2.501804e-06,6.966229e-10,0.001834,0.018358,2002
3,6.525,4.433100e+14,9.815350e+14,9.476020e+14,5.650410e+14,1.337090e+14,5.945700e+13,1.086090e+14,1.727270e+14,2.245720e+14,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.004670,0.018973,1052
4,6.525,2.093670e+14,4.521590e+14,4.341860e+14,2.476010e+14,5.047690e+13,2.374190e+13,5.077140e+13,9.692880e+13,8.705620e+13,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.004670,0.018973,1362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,1.500,2.770090e+14,7.704920e+14,8.418140e+14,5.382790e+14,1.577620e+14,6.232850e+13,1.527530e+14,2.087840e+14,2.461210e+14,...,2.187053e-07,4.272059e-07,8.870959e-08,3.120413e-06,2.401510e-07,6.803795e-07,2.836366e-10,0.003398,0.018730,1164
1496,6.525,1.803060e+14,3.622910e+14,4.170750e+14,1.906650e+14,4.197620e+13,1.168400e+13,5.086290e+13,7.012140e+13,8.326840e+13,...,2.153221e-07,6.222837e-07,8.307816e-08,4.076010e-06,2.412625e-07,9.813019e-07,2.918326e-10,0.003032,0.018643,2714
1497,1.500,4.899320e+14,1.048820e+15,1.117200e+15,5.839460e+14,1.233350e+14,3.624440e+13,1.141980e+14,1.620990e+14,2.309870e+14,...,1.960213e-08,4.229460e-08,8.981434e-08,1.094141e-06,1.969685e-07,2.028209e-07,7.093267e-11,0.004038,0.018850,2538
1498,6.525,4.575800e+14,1.020450e+15,1.088870e+15,4.951050e+14,9.643480e+13,2.739940e+13,5.878540e+13,1.073740e+14,1.415540e+14,...,4.334218e-07,1.090545e-06,8.667526e-08,5.494732e-06,2.400658e-07,1.316647e-06,3.947617e-10,0.002740,0.018622,2411


In [9]:
file_names = os.listdir("burnup_models/")
concentration_output = []
for i in range(1,1+iterations):
    try:
        concentration_output += extract_from_bumat(f"burnup_models/burnup{i}.serpent.bumat1")
    except:
        print(f"Could not find burnup_models/burnup{i}.serpent.bumat1")

In [10]:
post_burn_conc_df = pd.DataFrame(concentration_output).fillna(0)

In [11]:
features_df.iloc[0:len(post_burn_conc_df)].to_csv("training_data/burnup_features.csv")
post_burn_conc_df.to_csv("training_data/burnup_target.csv")